In [0]:
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn import preprocessing , neighbors ,svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [0]:
#ploting one from each genre in a spectogram

genres='blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:     
    song=f'./genres/{g}/{g}.00000.wav'
    x, sr=librosa.load(song, mono=True, duration=5)
    plt.figure(figsize=(10, 3))
    plt.title(g)
    librosa.display.waveplot(x, sr)

In [0]:
#extracting features to .csv file

header='filename chroma_stft rms spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header+=f' mfcc{i}'
header+=' label'
header=header.split()

file=open('data.csv', 'w', newline='')
with file:
    writer=csv.writer(file)
    writer.writerow(header)
genres='blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for f in os.listdir(f'./genres/{g}'):
        song=f'./genres/{g}/{f}'
        x, sr=librosa.load(song,mono=True,duration=30)
        chroma_stft=librosa.feature.chroma_stft(x,sr)
        rms=librosa.feature.rms(x)
        spec_cent=librosa.feature.spectral_centroid(x,sr)
        spec_bw=librosa.feature.spectral_bandwidth(x,sr)
        rolloff=librosa.feature.spectral_rolloff(x,sr)
        zcr=librosa.feature.zero_crossing_rate(y)
        mfcc=librosa.feature.mfcc(x,sr)
        to_append=f'{filename} {np.mean(chroma_stft)} {np.mean(rms)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append+=f' {np.mean(e)}'
        to_append+=f' {g}'
        file=open('data.csv', 'a', newline='')
        with file:
            writer=csv.writer(file)
            writer.writerow(to_append.split())


In [3]:
data=pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.wav,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596748,121.557297,-19.158827,42.351032,-6.376458,18.618876,-13.697912,15.344631,-12.285267,10.980493,-8.324325,8.810669,-3.667369,5.751691,-5.162763,0.750948,-1.691938,-0.409953,-2.300209,1.219929,blues
1,blues.00000.wav,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556793,124.006721,8.930560,35.874687,2.916037,21.523726,-8.554703,23.358671,-10.103617,11.903744,-5.560388,5.376803,-2.239120,4.216963,-6.012273,0.936110,-0.716537,0.293876,-0.287431,0.531573,blues
2,blues.00000.wav,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754387,140.459915,-29.109968,31.689013,-13.987036,25.754759,-13.649587,11.629272,-11.780589,9.706443,-13.123111,5.789265,-8.905224,-1.083720,-9.218360,2.455806,-7.726901,-1.815723,-3.433434,-2.226821,blues
3,blues.00000.wav,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431152,150.099213,5.647593,26.871927,1.754461,14.238345,-4.830883,9.297966,-0.757742,8.149013,-3.196314,6.087676,-2.476421,-1.073890,-2.874778,0.780977,-3.316932,0.637981,-0.619690,-3.408233,blues
4,blues.00000.wav,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266037,126.198807,-35.605450,22.153301,-32.489265,10.864513,-23.357929,0.503118,-11.805832,1.206806,-13.083821,-2.806384,-6.934123,-7.558618,-9.173553,-4.512165,-5.453538,-0.924161,-4.409333,-11.703781,blues


In [0]:
data=data.drop(['filename'],axis=1)

In [0]:
#encoding label and standardizing the data
genrelist=data.iloc[:, -1]
encoder=LabelEncoder()
y=encoder.fit_transform(genrelist)
scaler=StandardScaler()
x=scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [6]:
def scale_data(df, column_list):
    for column in column_list:
        df[column] = df[column].astype('float')
        encoder = preprocessing.StandardScaler()
        df[column] = encoder.fit_transform(df[column].values.reshape(-1,1))
        print("The ",column, "is encoded")
    return(df)
  
def encode_categorical(df, column_list):
    for column in column_list:
        df[column] = df[column].astype('str')
        encoder = preprocessing.LabelEncoder()
        encoded_list = encoder.fit_transform(df[column])
#         print(encoded_list)
#         print(len(encoded_list))
        encoded_series = pd.Series(encoded_list)
        df[column] = encoded_series
        print("The ", column, "is encoded ")
    return(df)

column=['chroma_stft', 'rms', 'spectral_centroid', 'spectral_bandwidth',
       'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4',
       'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11',
       'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18',
       'mfcc19', 'mfcc20']
data = scale_data(data,column)
data = encode_categorical(data, ['label'])
data.head()

The  chroma_stft is encoded
The  rms is encoded
The  spectral_centroid is encoded
The  spectral_bandwidth is encoded
The  rolloff is encoded
The  zero_crossing_rate is encoded
The  mfcc1 is encoded
The  mfcc2 is encoded
The  mfcc3 is encoded
The  mfcc4 is encoded
The  mfcc5 is encoded
The  mfcc6 is encoded
The  mfcc7 is encoded
The  mfcc8 is encoded
The  mfcc9 is encoded
The  mfcc10 is encoded
The  mfcc11 is encoded
The  mfcc12 is encoded
The  mfcc13 is encoded
The  mfcc14 is encoded
The  mfcc15 is encoded
The  mfcc16 is encoded
The  mfcc17 is encoded
The  mfcc18 is encoded
The  mfcc19 is encoded
The  mfcc20 is encoded
The  label is encoded 


,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,-0.351748,-0.010723,-0.583303,-0.456037,-0.486166,-0.491967,0.308252,0.702674,-0.472090,0.363653,-0.428098,0.336699,-0.862290,0.499553,-0.638827,0.409660,-0.337931,0.646276,0.183186,0.792921,-0.265290,-0.086750,0.500287,-0.237191,0.007611,0.603498,0
1,-0.461466,-0.533266,-0.939066,-0.387668,-0.649869,-1.138220,-0.629608,0.780890,0.823297,-0.025114,0.332557,0.582147,-0.344691,1.265798,-0.375344,0.526026,0.067600,0.134824,0.414751,0.486404,-0.439656,-0.046294,0.714737,-0.055190,0.543824,0.424035,0
2,-0.184483,0.680012,-0.907419,-0.941680,-0.972811,-0.653772,0.536253,1.306278,-0.931003,-0.276376,-1.051076,0.939653,-0.857427,0.144317,-0.577876,0.249079,-1.042019,0.196257,-0.666035,-0.572254,-1.097722,0.285748,-0.826553,-0.600707,-0.294285,-0.295113,0
3,0.319725,0.156317,-1.581483,-1.228392,-1.516328,-1.681950,-0.548502,1.614083,0.671898,-0.565540,0.237474,-0.033438,0.030065,-0.078585,0.753379,0.052781,0.414461,0.240704,0.376277,-0.570291,0.204331,-0.080189,0.143018,0.033792,0.455308,-0.603122,0
4,-0.858124,-0.599604,-0.511931,-0.939405,-0.629459,-0.051111,-0.157576,0.850888,-1.230552,-0.848794,-2.565611,-0.318513,-1.834449,-0.919485,-0.580925,-0.822213,-1.036254,-1.084009,-0.346457,-1.865428,-1.088525,-1.236700,-0.326734,-0.370160,-0.554268,-2.765875,0


In [7]:
import pandas_profiling as pp
pp.ProfileReport(data)

Number of variables,27
Number of observations,1000
Total Missing (%),0.0%
Total size in memory,211.1 KiB
Average record size in memory,216.1 B
Numeric,25
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,2


In [0]:
#splitting the data

x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=0.2)

In [9]:
#KNN

clf=neighbors.KNeighborsClassifier()
clf.fit(x_train,y_train)
accuracy = clf.score(x_test, y_test)
print(accuracy)

0.62


In [10]:
#SVM

clf=svm.SVC()
clf.fit(x_train,y_train)
accuracy = clf.score(x_test, y_test)
print(accuracy)

0.665


In [11]:
#Decision tree

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion = 'entropy')
clf.fit(x_train, y_train)
accuracy = clf.score(x_test, y_test)
print(accuracy)

0.485


In [13]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(x_train, y_train)
accuracy = clf.score(x_test, y_test)
print(accuracy)

0.615
